### Sensitivity analysis --> Sobol 
Defining parameters to vary and find variance on parameters outcome --> damage important to all actors
Finding variance that explains that damage is highest in other scenarios --> RfR & damage is low --> dike heightening 

In [10]:
## Importing all functions

%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome,
                           perform_experiments, ema_logging,Policy)

from ema_workbench import Samplers

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem

from ema_workbench.analysis.scenario_discovery_util import RuleInductionType

from ema_workbench.util import ema_logging

from dike_model_function import DikeNetwork

# exp, out = evaluator.perform_experiments(...,uncertainty_sampling=Samplers.SOBOL)

from SALib.analyze import sobol


In [ ]:
from ema_workbench import (
    Model,
    RealParameter,
    IntegerParameter,
    CategoricalParameter,
    ScalarOutcome,
    Samplers,
    perform_experiments
)
from ema_workbench.em_framework import Samplers
from ema_workbench.em_framework.salib_samplers import (get_SALib_problem)
from dike_model_function import DikeNetwork
from ema_workbench.util import ema_logging
from SALib.analyze import sobol

# Define the sum_over function
def sum_over(*args):
    numbers = []
    for entry in args:
        try:
            value = sum(entry)
        except TypeError:
            value = entry
        numbers.append(value)
    return sum(numbers)

# Configure logging
ema_logging.log_to_stderr(ema_logging.INFO)

# Define the function to get the model for the given problem formulation
def get_model_for_problem_formulation(problem_formulation_id):
    function = DikeNetwork()
    dike_model = Model("dikesnet", function=function)

    Real_uncert = {"Bmax": [30, 350], "pfail": [0, 1]}  # m and [.]
    cat_uncert_loc = {"Brate": (1.0, 1.5, 10)}
    cat_uncert = {f"discount rate {n}": (1.5, 2.5, 3.5, 4.5) for n in function.planning_steps}
    Int_uncert = {"A.0_ID flood wave shape": [0, 132]}

    uncertainties = []

    for uncert_name in cat_uncert.keys():
        categories = cat_uncert[uncert_name]
        uncertainties.append(CategoricalParameter(uncert_name, categories))

    for uncert_name in Int_uncert.keys():
        uncertainties.append(IntegerParameter(uncert_name, Int_uncert[uncert_name][0], Int_uncert[uncert_name][1]))

    dike_height_levers = []
    for dike in function.dikelist:
        for uncert_name in Real_uncert.keys():
            name = f"{dike}_{uncert_name}"
            lower, upper = Real_uncert[uncert_name]
            uncertainties.append(RealParameter(name, lower, upper))

        for uncert_name in cat_uncert_loc.keys():
            name = f"{dike}_{uncert_name}"
            categories = cat_uncert_loc[uncert_name]
            uncertainties.append(CategoricalParameter(name, categories))
        
        for n in function.planning_steps:
            name = f"{dike}_DikeIncrease {n}"
            dike_height_levers.append(IntegerParameter(name, 0, 10))
        
    dike_model.uncertainties = uncertainties

    dike_height_levers.append(IntegerParameter("EWS_DaysToThreat", 0, 4))  # days
    # Set levers: No RfR, dike heightening
    dike_model.levers = dike_height_levers

    # Define the outcomes
    outcomes = [
        ScalarOutcome('Total Costs', kind=ScalarOutcome.MINIMIZE, function=sum_over, variable_name=[
            f"{dike}_Expected Annual Damage" for dike in function.dikelist] +
            [f"{dike}_Dike Investment Costs" for dike in function.dikelist] +
            ["RfR Total Costs"] 
        ),
        ScalarOutcome('Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE, function=sum_over, variable_name=[
            f"{dike}_Expected Number of Deaths" for dike in function.dikelist]
        ),
        ScalarOutcome('Expected Annual Damage', kind=ScalarOutcome.MINIMIZE, function=sum_over, variable_name=[
            f"{dike}_Expected Annual Damage" for dike in function.dikelist]),
        ScalarOutcome('Dike Investment Costs', kind=ScalarOutcome.MINIMIZE, function=sum_over, variable_name=[f"{dike}_Dike Investment Costs" for dike in function.dikelist]),
        ScalarOutcome('RfR Total Costs', kind=ScalarOutcome.MINIMIZE, function=sum_over, variable_name=["RfR Total Costs"] )
    ]
    
    # 
    # # Define the levers
    # levers = [
    #     # Range of dike heightening:
    #     IntegerParameter("DikeIncrease", 0, 10),  # dm
    # 
    #     # Series of five Room for the River projects:
    #     *[CategoricalParameter(f"{project_id}_RfR", [0, 1]) for project_id in range(0, 5)],
    # 
    #     # Time of warning: 0, 1, 2, 3, 4 days ahead from the flood
    #     IntegerParameter("EWS_DaysToThreat", 0, 4)  # days
    # ]

    # Assign uncertainties, outcomes, and levers to the model
    # dike_model.uncertainties = uncertainties
    dike_model.outcomes = outcomes
    
    # dike_model.levers = levers 

    return dike_model

# Define policies to test different dike height increases
policies = [Policy(f"DikeIncrease_{i}", **{f"{dike}_DikeIncrease {n}": i for dike in dike_model.function.dikelist for n in dike_model.function.planning_steps}, **{"EWS_DaysToThreat": 2}) for i in range(1, 11, 2)]



# Get the model for a specific problem formulation
problem_formulation_id = 6  # Change this to the desired problem formulation
dike_model = get_model_for_problem_formulation(problem_formulation_id)

# Get SALib problem
problem = get_SALib_problem(dike_model.uncertainties)

# Print the problem definition
print(problem)


In [ ]:
n_exp = 750

experiments_sobol, outcomes_sobol = perform_experiments(dike_model, scenarios=n_exp,
                                                        policies=policies,
                                                        uncertainty_sampling=Samplers.SOBOL)

dike_final_sobol = outcomes_sobol["Total Costs"][:,0,-1]
dike_mean_sobol = np.mean(outcomes_sobol["Total Costs"][:,0,:],axis=1)
dike_std_sobol = np.std(outcomes_sobol["Total Costs"][:,0,:],axis=1)

Si = sobol.analyze(problem, dike_mean_sobol, calc_second_order=True, print_to_console=True)


In [ ]:
import pandas as pd
experiments_sobol.to_csv('experiments_sobol.csv')
outcomes_df = pd.DataFrame(outcomes_sobol)
outcomes_df.to_csv('outcomes_sobol.csv', index=False)

In [ ]:
dike_final_sobol = outcomes_sobol["Total Costs"]
dike_mean_sobol = np.mean(outcomes_sobol["Total Costs"])
dike_std_sobol = np.std(outcomes_sobol["Total Costs"])

# Check the shape of the data
print(f"Shape of outcomes_sobol['Total Costs']: {outcomes_sobol['Total Costs'].shape}")

# Ensure we use the correct number of samples
N = outcomes_sobol['Total Costs'].shape[0]

# Sobol analysis
try:
    Si = sobol.analyze(problem, outcomes_sobol['Total Costs'], calc_second_order=True, print_to_console=True)
    print("Sobol analysis completed successfully")
except Exception as e:
    print(f"Error during Sobol analysis: {e}")



In [ ]:
Si_filter = {k:Si[k] for k in ['ST','ST_conf','S1','S1_conf']}
Si_df = pd.DataFrame(Si_filter, index=problem['names'])

sns.set_style('white')
fig, ax = plt.subplots(1)

indices = Si_df[['S1','ST']]
err = Si_df[['S1_conf','ST_conf']]

indices.plot.bar(yerr=err.values.T,ax=ax)
fig.set_size_inches(8,6)
fig.subplots_adjust(bottom=0.3)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from SALib.analyze import sobol

# Check the shape of the data
print(f"Shape of outcomes_sobol['Total Costs']: {outcomes_sobol['Total Costs'].shape}")

# Perform Sobol analysis
n_exp = outcomes_sobol['Total Costs'].shape[0]

# DataFrame to store Sobol indices
s_data = pd.DataFrame(index=problem['names'], columns=np.arange(20, n_exp, 50) * (2 * problem['num_vars'] + 2))

try:
    # Using tqdm to show progress
    for j in tqdm(s_data.columns, desc="Sobol Analysis Progress"):
        # Ensure Y is properly indexed
        scores = sobol.analyze(problem, outcomes_sobol['Total Costs'][0:j], calc_second_order=True, print_to_console=False)
        s_data.loc[:, j] = scores['ST']
    print("Sobol analysis completed successfully")
except IndexError as e:
    print(f"IndexError during Sobol analysis: {e}")
except Exception as e:
    print(f"Error during Sobol analysis: {e}")

# Print the resulting DataFrame
print(s_data)


In [ ]:
fig, ax = plt.subplots(1)

s_data.T.plot(ax=ax)
ax.set_xlabel('Samples')
ax.set_ylabel('Total index (ST)')
plt.show()

##Extra Trees##

In [39]:
from ema_workbench.analysis import feature_scoring

In [40]:
dike_final_sobol = outcomes_sobol["Total Costs"]
dike_mean_sobol = np.mean(outcomes_sobol["Total Costs"])
dike_std_sobol = np.std(outcomes_sobol["Total Costs"])

In [49]:
res = feature_scoring.get_ex_feature_scores(experiments_sobol, dike_final_sobol,
                                      max_features=0.6, mode=RuleInductionType.REGRESSION)[0]
scores

C:\Users\Joost\PycharmProjects\venv\lib\site-packages\sklearn\utils\_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


,33500
0,
A.1_pfail,0.639911
A.3_pfail,0.189355
A.5_pfail,0.020151
discount rate 0,0.019271
A.2_pfail,0.014930
discount rate 1,0.013387
discount rate 2,0.009444
Unnamed: 0,0.005047
policy,0.005029


In [ ]:
import pandas as pd
import numpy as np
from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.feature_scoring import RuleInductionType
from tqdm import tqdm  # Import tqdm for the progress bar

# Assuming outcomes_sobol and experiments_sobol are loaded correctly as previously mentioned

# Ensure dike_mean_sobol is correctly calculated as an array
dike_mean_sobol = np.mean(outcomes_sobol["Total Costs"].reshape(-1, 1), axis=1)

combined_scores = []

# Adding a progress bar to the loop
for j in tqdm(range(100, experiments_sobol.shape[0], 100), desc="Feature Scoring Progress"):
    scores = feature_scoring.get_ex_feature_scores(experiments_sobol.iloc[0:j, :],
                                                   dike_mean_sobol[0:j],
                                                   max_features=0.6,
                                                   mode=RuleInductionType.REGRESSION)[0]
    scores.columns = [j]
    combined_scores.append(scores)

# Combine the scores DataFrames
combined_scores_df = pd.concat(combined_scores, axis=1)

# Optionally, print or save the combined_scores_df
print(combined_scores_df)


Feature Scoring Progress:   0%|          | 0/1499 [00:00<?, ?it/s]C:\Users\Joost\PycharmProjects\venv\lib\site-packages\sklearn\utils\_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)
Feature Scoring Progress:   0%|          | 1/1499 [00:00<04:17,  5.82it/s]C:\Users\Joost\PycharmProjects\venv\lib\site-packages\sklearn\utils\_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)
Feature Scoring Progress:   0%|          | 2/1499 [00:00<04:03,  6.15it/s]C:\Users\Joost\PycharmProjects\venv\lib\site-packages\sklearn\utils\_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)
Feature Scoring Progress:   0%|          | 3/1499 [00:00<04:41,  5.31it/s]C:\Users\Joost\PycharmProjects\venv\lib\site-packages\sklearn\utils\_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  ret

In [ ]:
fig, ax = plt.subplots(1)

combined_scores.T.plot(ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.set_xlabel('Samples')
ax.set_ylabel('feature scores')
plt.show()

In [ ]:
cleaned_experiments = experiments.drop(columns=[l.name for l in lake_model.levers])

scores = feature_scoring.get_feature_scores_all(cleaned_experiments,results)

In [ ]:
import seaborn as sns
cleaned_experiments = experiments.drop(columns=[l.name for l in lake_model.levers])

for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    subset_results = {k:v[logical] for k,v in results.items()}
    scores = feature_scoring.get_feature_scores_all(cleaned_experiments[logical],
                                                   subset_results)
    sns.heatmap(scores, annot=True, cmap='viridis')
    plt.show()